Como sempre, o primeiro passo é importar os dados do CSV por meio do Pandas

In [ ]:
import pandas as pd
dataFrame = pd.read_csv("/content/Market_Basket_Optimization.csv",header=None)
print("Quantidade de linhas importadas:",len(dataFrame))
print("Quantidade de colunas do dataframe:", len(dataFrame.columns))

Quantidade de linhas importadas: 7501
Quantidade de colunas do dataframe: 20


Como sempre de novo, após importar o data frame, convém fazer uma análise exploratória dos dados importados.

In [ ]:
dataFrame.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Para trabalhar com associação não se utiliza o data  frame diretamente. A associação trabalha bem com dados organizados em listas. Para alimentar a lista, faremos dois loops, um para percorrer todas as linhas do data frame e outro para percorrer todas as colunas em cada linha.

In [ ]:
#criação da lista
transactions = []
#criação de lista para armazenar os valores de uma transacao
oneTransact = []
# popular a lista com os valores do data frame
for i in range(0, 7501):
  for j in range(0,20):
    oneTransact.append(str(dataFrame.values[i,j]))
    
  #print("Transacao", i, "Valores:", oneTransact)
  transactions.append(oneTransact)
  oneTransact=[]
transactions


[['shrimp',
  'almonds',
  'avocado',
  'vegetables mix',
  'green grapes',
  'whole weat flour',
  'yams',
  'cottage cheese',
  'energy drink',
  'tomato juice',
  'low fat yogurt',
  'green tea',
  'honey',
  'salad',
  'mineral water',
  'salmon',
  'antioxydant juice',
  'frozen smoothie',
  'spinach',
  'olive oil'],
 ['burgers',
  'meatballs',
  'eggs',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan'],
 ['chutney',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan'],
 ['turkey',
  'avocado',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan'],
 ['mineral water',
  'milk',
  'energy bar',
  'whole wheat rice',
  'green tea',
  'nan',
  'nan',
  'nan',
 

Agora que s dados estão preparados, vamos iniciar a tarefa de associação.

In [ ]:
!pip install apyori

  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=d240d274758d7f46d8e1eb5a6eef3a2e90833e51705861d02be52d6d8cff62ad
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


In [ ]:
from apyori import apriori

Agora que o Apriori foi importado, vamos treinar o modelo.

In [ ]:
rule = apriori(transactions=transactions,min_support=0.003, min_confidence=0.2, min_lift=1.1)

In [ ]:
output = list(rule)
output

[RelationRecord(items=frozenset({'burgers', 'almonds'}), support=0.005199306759098787, ordered_statistics=[OrderedStatistic(items_base=frozenset({'almonds'}), items_add=frozenset({'burgers'}), confidence=0.25490196078431376, lift=2.923577382023146)]),
 RelationRecord(items=frozenset({'chocolate', 'almonds'}), support=0.005999200106652446, ordered_statistics=[OrderedStatistic(items_base=frozenset({'almonds'}), items_add=frozenset({'chocolate'}), confidence=0.29411764705882354, lift=1.7950988369310295)]),
 RelationRecord(items=frozenset({'eggs', 'almonds'}), support=0.006532462338354886, ordered_statistics=[OrderedStatistic(items_base=frozenset({'almonds'}), items_add=frozenset({'eggs'}), confidence=0.3202614379084967, lift=1.7821076007059597)]),
 RelationRecord(items=frozenset({'french fries', 'almonds'}), support=0.004399413411545127, ordered_statistics=[OrderedStatistic(items_base=frozenset({'almonds'}), items_add=frozenset({'french fries'}), confidence=0.21568627450980393, lift=1.261

O resultado exibido pelo Apriori é bagunçado. Para melhorar a nossa análise, vamos estruturar o resultado em dataFrame

In [ ]:
#output = list(rule) # returns a non-tabular output
# putting output into a pandas dataframe
def inspect(output):
    lhs         = [tuple(result[2][0][0])[0] for result in output]
    rhs         = [tuple(result[2][0][1])[0] for result in output]
    support    = [result[1] for result in output]
    confidence = [result[2][0][2] for result in output]
    lift       = [result[2][0][3] for result in output]
    return list(zip(lhs, rhs, support, confidence, lift))
resultadoFormatado = inspect(output)
resultadoFormatado
output_DataFrame = pd.DataFrame(resultadoFormatado, columns = ['Left_Hand_Side', 'Right_Hand_Side', 'Support', 'Confidence', 'Lift'])

In [ ]:
output_DataFrame

,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
0,almonds,burgers,0.005199,0.254902,2.923577
1,almonds,chocolate,0.005999,0.294118,1.795099
2,almonds,eggs,0.006532,0.320261,1.782108
3,almonds,french fries,0.004399,0.215686,1.261983
4,almonds,green tea,0.005066,0.248366,1.879913
...,...,...,...,...,...
1610,ground beef,mineral water,0.003066,0.211009,3.532991
1611,ground beef,mineral water,0.003066,0.261364,4.376091
1612,mineral water,nan,0.003333,0.390625,2.243551
1613,mineral water,nan,0.003066,0.389831,2.238988


Vimos que o lift é uma boa medida para definir a melhor combinação. Se o lift for maior que um, é considerado bom. Entretanto o melhor lift vai ser o maior da base.

In [ ]:
output_DataFrame.nlargest(n=30, columns = ['Lift'])

,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
1425,frozen vegetables,mineral water,0.003066,0.383333,7.987176
1604,frozen vegetables,mineral water,0.003066,0.383333,7.987176
1424,olive oil,mineral water,0.003333,0.294118,6.128268
1603,olive oil,mineral water,0.003333,0.294118,6.128268
1553,mineral water,olive oil,0.003866,0.402778,6.128268
989,mineral water,olive oil,0.003866,0.402778,6.115863
888,tomato sauce,ground beef,0.003066,0.216981,5.535971
1505,tomato sauce,ground beef,0.003066,0.216981,5.535971
760,fromage blanc,honey,0.003333,0.245098,5.178818
157,fromage blanc,honey,0.003333,0.245098,5.164271
